### Analyse search terms on the e-commerce web server


In [2]:
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 MB 1.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 35.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.2-py2.py3-none-any.whl size=311619841 sha256=85d36c9d231190cc49f0a4ee49795398bab7c8ca78fe0877f0c4efebe47f2de2
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/c3/8a/ac/cd39777597318310141c8a783c06f516815a66194f100f96b6
Successfully built pyspark


In [3]:
import findspark
findspark.init()


from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("e-commerce").getOrCreate()

24/02/06 14:00:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Downloading The search term dataset from the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2024-02-06 14:00:33--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.005s  

2024-02-06 14:00:33 (44.2 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [5]:
df = spark.read.csv('searchterms.csv', header=True, inferSchema=True)

In [6]:
row = df.count()
column = len(df.columns)
print("rows is "+str(row)+" and columns is "+str(column))

rows is 10000 and columns is 4


In [7]:
# Printing the top 5 rows

In [8]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [9]:
# The datatype of the column searchterm?

In [10]:
df.printSchema()

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [11]:
# Finding the number of times the term `gaming laptop` was searched?

In [12]:
df.filter("searchterm = 'gaming laptop'").count()

499

In [13]:
# Printing the top 5 most frequently used search terms?

In [14]:
df.groupBy('searchterm').count().sort("count", ascending=False).show(5)

[Stage 8:====================================================>  (192 + 8) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [15]:
# Downloading the pretrained sales forecasting model at the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

--2024-02-06 14:01:21--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2024-02-06 14:01:21 (8.10 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [33]:
# Loading the sales forecast model

In [34]:
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load("sales_prediction.model")

In [35]:
# Using the sales forecast model, predict the sales for the year of 2023.

In [36]:
from pyspark.ml.feature import VectorAssembler
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "searchterm"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','searchterm')
    predictions = model.transform(__)
    predictions.select('prediction').show()
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



24/02/06 14:16:36 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/02/06 14:16:36 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
